# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
### By *Ricardo Picatoste*
***
Here I describe the pipeline I created step by step.

More comments on the code and how it has been done are explained in the writeup.md file.

---

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
#from functions import *
import os
import math
%matplotlib inline

## Read in an Image

In [ ]:
# Get the image
image_name = "solidWhiteCurve.jpg"
image = mpimg.imread(image_name)

# printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow( image ) 


## Build a Lane Finding Pipeline

I will perform the pipeline on the example image step by step. For this I start by defining all the functions that are used in the pipeline.

NOTE: The coefficients used along the pipeline have been hard coded for this project. 

In [ ]:
# Parameters for the different pipeline stages.
# Parameters of the detection
ker_size = 5
low_threshold = 50
high_threshold = 150

# Parameters to select ROI as fraction of the image.
fraction_top = 1/100*60     
fraction_left_top = 1/100*40
fraction_left_bottom = 1/100*10
fraction_right_top = 1/100*60
fraction_right_bottom = 1/100*90

# Define the Hough transform parameters
rho = 1*1.0            # distance resolution in pixels of the Hough grid
theta = np.pi/180 *1.0 # angular resolution in radians of the Hough grid
threshold = 3          # minimum number of votes (intersections in Hough grid cell)
min_line_length = 20   # minimum number of pixels making up a line
max_line_gap = 150     # maximum gap in pixels between connectable line segments



First the image is converted to gray and blurred

In [ ]:

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel """
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)    
    
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

im_gray = grayscale( image )
im_blur = gaussian_blur( im_gray, ker_size )

plt.imshow( im_blur ) 
print("image gray and blurred")

Now we apply the Canny transform to obtain the gradient of the image, and the result is cropped to have only the region of interest, that where the lane lines will mostly be.

In [ ]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.  
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on
    #the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

im_canny = canny( im_blur , low_threshold, high_threshold)

# This time we are defining a four sided polygon to mask
imshape = image.shape

# The vertices of the region of interest are created from the image shape
# information and the fractional parameters selected at the beginning. This 
# means that the ROI is chosen as a fraction of the original image, centered in
# the bottom middle of the image. 
bottom = imshape[0]
top = imshape[0]*fraction_top
top_left = imshape[1]*fraction_left_top
top_right = imshape[1]*fraction_right_top
bottom_left = imshape[1]*fraction_left_bottom
bottom_right = imshape[1]*fraction_right_bottom 

vertices = np.array([[  (bottom_left,   bottom),\
                        (top_left,      top), \
                        (top_right,     top), \
                        (bottom_right,  bottom)]], \
                        dtype=np.int32)

im_cropped = region_of_interest( im_canny, vertices )

plt.imshow( im_cropped ) 


Now the Hough transform is applied. Once applied, the draw_lines funtion is also used. This function will try to get, from the lines found, the one most likely representing the left lane and the same for the right lane.

In [ ]:

def line_get_distance(line):
    
    x1,y1,x2,y2 = line[0]
    return math.sqrt( (x2-x1)**2 + (y2-y1)**2 )
    
def line_get_slope_and_offset(x1,y1,x2,y2):
    
    if x1 == x2:
        m0 = math.inf
    else:
        m0 = (y2-y1)/(x2-x1)

    b0 = y1 - m0*x1
    
    return m0, b0

def line_get_x(m, b, y):
    if m == 0.0:
        return math.nan
    if m == math.inf:
        return 0.0 
    
    x = (y-b)/m        
    return x

def draw_lines(img, lines, roi_vertices, color=[255, 0, 0], thickness=2):
    """
    NOTE: choose the best lines represting the lanes and plot them. 
    
    """
        
    # Some values are obtained from the region of interest: the top and bottom
    # limits, the center of the image.
    middle_of_image = int( (roi_vertices[0][0][0]+roi_vertices[0][1][0]) / 2.0 )
    ybottom = roi_vertices[0][0][1]        
    ytop = roi_vertices[0][1][1]
    
    # Initialize values to obtain the best lines (longest ones).
    left_max_distance = 0
    right_max_distance = 0
    left_max_index = -1
    right_max_index = -1
    
    # Select the longest lines, one for the right and one for the left.
    for i in range(len(lines)):
        x1,y1,x2,y2 = lines[i][0]
        
        distance = line_get_distance(lines[i])
        
        m0, b0 = line_get_slope_and_offset(x1,y1,x2,y2)
        # Get the x where the line starts at the bottom to help deciding left 
        # or right
        xbottom = line_get_x(m0, b0, ybottom)
        # Lines below a threshold degrees and above 180-threshold will be 
        # dropped (too horizontal). 
        threshold = 0.5
        # Process right
        if m0 >= threshold and line_get_x(m0, b0, ybottom) > middle_of_image: 
            if(distance > right_max_distance):
                right_max_distance = distance
                right_max_index = i
        elif m0 <= -threshold and line_get_x(m0, b0, ybottom) < middle_of_image: 
            if(distance > left_max_distance):
                left_max_distance = distance
                left_max_index = i
                
    # Convert lines from the best found to complete lines.
    for i in [left_max_index, right_max_index]:
        if i == -1:
            continue
        
        x1,y1,x2,y2 = lines[i][0]
        # Get standard equation
        m0, b0 = line_get_slope_and_offset(x1,y1,x2,y2)
        
        # Obtain the points corresponding to the bottom and top
        if m0 == math.inf:
            xbottom = x1
            xtop = x1
        elif m0 == 0.0:
            xbottom = x1
            xtop = x1
        else:
            xbottom = int((ybottom - b0) / m0)
            xtop = int((ytop - b0) / m0)
            
        cv2.line(img, (xbottom, ybottom), (xtop, ytop), [0, 255, 0], thickness*3)

def hough_lines(img, roi_vertices, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    roi_vertices is the region of interest. It will be used to plot the lanes
    will full extent from the best lines found for them. 
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, \
                            np.array([]), \
                            minLineLength = min_line_len, \
                            maxLineGap = max_line_gap )
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, roi_vertices)
    return line_img
    

im_hough = hough_lines(im_cropped, vertices, rho, theta, threshold, min_line_length, max_line_gap)
    
plt.imshow( im_hough ) 
        

Finally the last step is to combine the lines obtained with the original image.

In [ ]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    
    return cv2.addWeighted(initial_img, α, img, β, λ)

im_final = weighted_img(im_hough, image, α=0.8, β=1., λ=0.)
plt.imshow( im_final ) 

All the mentioned steps are combined in a single function, called my_pipeline, which will receive a single image and give back the result being this the combination of the original image and the lanes found superimposed. 

In [ ]:
def my_pipeline(image, plot_all=0):
    
    # Parameters of the different pipeline stages
    # Parameters of the detection
    ker_size = 5
    low_threshold = 50
    high_threshold = 150

    # Parameters to select ROI as fraction of the image.
    fraction_top = 1/100*60     
    fraction_left_top = 1/100*40
    fraction_left_bottom = 1/100*10
    fraction_right_top = 1/100*60
    fraction_right_bottom = 1/100*90

    # Define the Hough transform parameters
    rho = 1*1.0            # distance resolution in pixels of the Hough grid
    theta = np.pi/180 *1.0 # angular resolution in radians of the Hough grid
    threshold = 3          # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20   # minimum number of pixels making up a line
    max_line_gap = 150     # maximum gap in pixels between connectable line segments

    if(plot_all):
        plt.imshow(image) 
        print("image")
        plt.show()
    
    im_gray = grayscale( image )
    
    if(plot_all):
        plt.imshow( im_gray ) 
        print("image gray")
        plt.show()
    
    im_blur = gaussian_blur( im_gray, ker_size )
    
    if(plot_all):
        plt.imshow( im_blur ) 
        print("image gray and blur")
        plt.show()
    
    im_canny = canny( im_blur , low_threshold, high_threshold)
    
    if(plot_all):
        plt.imshow( im_canny ) 
        print("canny")
        plt.show()
    
    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    
    bottom = imshape[0]
    top = imshape[0]*fraction_top
    top_left = imshape[1]*fraction_left_top
    top_right = imshape[1]*fraction_right_top
    bottom_left = imshape[1]*fraction_left_bottom
    bottom_right = imshape[1]*fraction_right_bottom 
                   
    vertices = np.array([[  (bottom_left,   bottom),\
                            (top_left,      top), \
                            (top_right,     top), \
                            (bottom_right,  bottom)]], \
                            dtype=np.int32)
    
    im_cropped = region_of_interest( im_canny, vertices )
    
    if(plot_all):
        plt.imshow( im_cropped ) 
        print("cropped")
        plt.show()
    
    # Hough transform
    # Run Hough on edge detected image
    im_hough = hough_lines(im_cropped, vertices, rho, theta, threshold, min_line_length, max_line_gap)
    
    if(plot_all):
        plt.imshow( im_hough ) 
        print("hough")
        plt.show()
    
    
    im_final = weighted_img(im_hough, image, α=0.8, β=1., λ=0.)
    if(plot_all):
        plt.imshow( im_final ) 
        print("Final image")
        plt.show()

    return im_final

image = mpimg.imread("solidYellowCurve2.jpg")
im_result = my_pipeline(image)
plt.imshow( im_result ) 


## Test on Videos

Now the solution is run on the videos:

- solidWhiteRight.mp4
- solidYellowLeft.mp4
- challenge.mp4

And the results saved in the folder results\



In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):
    result = my_pipeline(image)
    return result

directory = os.path.dirname("results/")
if not os.path.exists(directory):
    os.makedirs(directory)
        
# next video
white_output = 'results\white.mp4'
clip1 = VideoFileClip('input_videos\solidWhiteRight.mp4')
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)
print("First video done")

# next video
yellow_output = 'results\yellow.mp4'
clip2 = VideoFileClip('input_videos\solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)
print("second video done")

# next video
challenge_output = 'results\chal.mp4'
clip2 = VideoFileClip('input_videos\challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)
print("extra video done")

Play the first video inline, or from the results folder in the project folder.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Play the second video in the same way.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

Finally the same with the challenge video.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

## Writeup and Submission

More comments on the writeup file.
